# The Token Authentication Method

The token authentication method is critical to understand with Vault, as it is the underlying authentication method powering all other authentication methods. In essence every call to Vault is either unauthenticated (meaning a user does not need to be logged-in to perform the action), or authenticated via a Vault token. Thus all other authentication methods essentially provide unauthenticated login endpoints which translate some form of credential (certificates, LDAP credentials, etc) into a Vault token.

## Use Cases

In the vast majority of cases the token authentiation method should not be used directly. This is because it is very basic, and provides an authentication that is local to Vault. In professional setups, one will want to use a managed identity or any other existing identity to authenticate to Vault.

The only case where the token authentication method is commonly used is when authentication as a superuser using a root token. Nonetheless, due to the fact that tokens are the underlying logic for all other authentication methods, it is primordial to understand how this method works.

### Starting Vault

Let us start a Vault server. This will run Vault in the background and push the logs to `/tmp/vault.log`. If at any point in time the Vault crashes, this command will need to be used again to re-launch the Vault server.

For now, do not worry about the configuration with which we are starting the server. This will be covered in a separate module.

In [ ]:
nohup bash -c '
  vault server -dev -dev-root-token-id=root-token -dev-listen-address="0.0.0.0:8200"
' > /tmp/vault.log 2>&1 &
echo $! > /tmp/vault.pid

### Logging In

We can login using a token with the CLI in one of two ways:

1. The `vault login` command.
2. Setting the `VAULT_TOKEN` environment variable.
3. Fixing the Vault CLI cache by manually creating a `~/.vault-token` file.

Let us try the first one. Typically, this would be done interactively so that the token does not get stored in the shell history. Due to limitations of Jupyter Notebooks, we will provide the token directly on the command line.

Perform the following:

1. Login using `vault login <token>`.
2. Check the contents of the `~/.vault-token` file.

The root token in our case is `root-token`.

In [ ]:
# login
export VAULT_ADDR="http://127.0.0.1:8200"
vault login root-token

Note the metadata attached to the token. It has an accessor attached to it. This can be used to obtain information about the token without requiring knowledge of the token itself. On top of that, the token has an attached time-to-live to it. This is a finite duration after which the token becomes invalid. Here we have an exception with the root token, which is the only type of token allowed to be valid indefinitely. Finally, the token has policies attached to it, which grant permissions to perform some actions against the Vault.

Now check the token cache:

In [ ]:
cat ~/.vault-token

Essentially, the `vault login` command is just a convenience command which performs a token lookup (essentially retrieving the metadata of the token), and caching it on disk for subsequent CLI invokations.

**ATTENTION**: this means that if you perform a `vault login` on a shared machine, people having access to your home directory will be able to read the token you used to authenticate to Vault! The same is true for other login commands of the CLI for other authentication methods. This is inherent to the fact that the CLI needs to store authentication information between invokations.

### Lookups

Token lookups can be done using either the token itself, or its accessor. Try this out using the token (`root-token`) and the accessor which was printed in the login above. The commands to do this is `vault token lookup`, and you need to provide the `-accessor` flag if providing an accessor.

In [ ]:
vault token lookup ...

### Creating Tokens

Now let us create some tokens. This can be done using the `vault token create` command. Familiarise yourself a little with the possible options. Specifically those under "Command Options".

In [ ]:
vault token create -h

### Assigning Policies to Tokens

Let us create a token with the following two policies `"test"` and `"admin"`. Note that it does not really matter that the policies do not exist.

In [ ]:
vault token create ...

### More Complex Tokens

Note that there are quite a few options for the time-to-live (TTL). This can be a bit confusing initially. Generally:

- The TTL provides the initial validity span of a token.
- Tokens can however be renewed, providing additional time to live beyond the initial validity span.
- The maximum explicit TTL provides a cap to how much a token can be renewed beyond its initial validity span.
- In some cases, tokens should live "forever". Only root tokens have an infinite validity span though. However, using periodic tokens, one can create tokens that are valid forever, as long as the token is renewed within a set period.

Now let us create a token with only the policy `"test"` but which only valid for 15 minutes and is not renewable.

Create another token that is only valid for 5 minutes, but is renewable with a max TTL of 1 hour. Renew that token after some time with the `vault token renew` command. How is the new TTL of the token affected.

> NOTE: the `"default"` policy that is attached to tokens by default enables the tokens to renew themselves.

Finally create a periodic token with a period of 1 hour.

### Revoking Tokens

But what happens if a token is leaked. You can revoke tokens using the `vault token revoke` command. This can again be performed using the accessor or the token itself. Try this out on one of the tokens you previously created that is still valid. Then perform a lookup and see that the token no longer exists.

### Token Types

Note that the tokens you created all started with `hvs.`. This prefix stands for "HashiCorp Vault Service", because these tokens are Service tokens. Vault as three types of tokens:

- Service tokens (`hvs.`),
- Batch tokens (`hvb.`),
- Recovery tokens (`hvr.`).

Using batch tokens is slightly more advanced, and not necessary for most use cases. They are mostly required if you are issuing massive amounts of tokens or require better performance. You can read more on token types here: https://developer.hashicorp.com/vault/docs/concepts/tokens#token-types-in-detail

> NOTE: the root token is a standard service token. The only reason it is `root-token` in our exercises and thus does not have a prefix is because we are using a development Vault server in these exercises and override the root token to make it easier to handle it in the exercises. In practice it would be a service token like any other.

If you want to give it a try, create a batch token:

## Cleaning Up

At the end of each module, you should clean up your Vault instance. This is done by shutting it down and wiping its database to restore its state.

In [ ]:
kill $(cat /tmp/vault.pid)
rm /tmp/vault.log
rm /tmp/vault.pid